## Get list of all recipes to scrape
<br> 


In [71]:
import boto3 #go get 'about_me' page 
from bs4 import BeautifulSoup as BS
import time
from urllib import urlencode  # TODO(Miles): Python 3
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
import random
import requests
import pymongo
import pandas as pd
import re
import collections
import pickle

In [3]:
client = pymongo.MongoClient('mongodb://localhost:27017/')
allrecipes_db = client.allrecipes
allrecipes_db.collection_names()

[u'members', u'webpages', u'community_link_pages', u'community_page']

In [5]:
members_coll = allrecipes_db.members

In [6]:
members_coll.find_one().keys()

[u'favorites_page',
 u'following_page',
 u'followers_dict',
 u'reviews_recipe_id_list',
 u'reviews_dict',
 u'reviews_page',
 u'madeits_recipe_id_list',
 u'followers_page',
 u'favorites_dict',
 u'madeit_page',
 u'following_dict',
 u'link',
 u'following_recipe_id_list',
 u'member_ID',
 u'favorites_recipe_id_list',
 u'followers_recipe_id_list',
 u'_id',
 u'recipes_page',
 u'madeits_dict']

In [13]:
# get list of all recipes
faves =  members_coll.find({"member_ID": {"$exists": True}}, ['favorites_recipe_id_list'])

In [23]:
members_coll.find({"member_ID": {"$exists": True}}).count()

815

In [14]:
faves = [a for a in faves]

In [15]:
len(faves)

815

In [19]:
faves = [a['favorites_recipe_id_list'] for a in faves]

In [20]:
faves[1]

[u'234645',
 u'234645',
 u'64559440',
 u'64559440',
 u'20906',
 u'20906',
 u'68806',
 u'68806',
 u'64168799',
 u'64168799',
 u'246102',
 u'246102',
 u'89948',
 u'89948',
 u'235983',
 u'235983',
 u'228238',
 u'228238',
 u'237347',
 u'237347',
 u'62937442',
 u'62937442',
 u'219483',
 u'219483',
 u'139471',
 u'139471',
 u'231624',
 u'231624',
 u'161522',
 u'161522',
 u'222705',
 u'222705',
 u'240511',
 u'240511',
 u'64360001',
 u'64360001',
 u'62177657',
 u'62177657',
 u'162392',
 u'162392']

In [21]:
faves = [item for sublist in faves for item in sublist]

In [25]:
len(set(faves))

7105

In [26]:
len(faves)

22442

In [28]:
1.*len(faves)/len(set(faves))*1.0

3.1586206896551725

In [32]:
favorite_recipes_counter = collections.Counter(faves)

In [34]:
favorite_recipes_counter.most_common(10)

[(u'219164', 50),
 (u'162760', 42),
 (u'10813', 38),
 (u'236609', 38),
 (u'23600', 38),
 (u'13116', 36),
 (u'14685', 34),
 (u'12409', 32),
 (u'143082', 32),
 (u'55860', 32)]

In [35]:
# get list of all reviewed recipes
reviewed =  members_coll.find({"member_ID": {"$exists": True}}, ['reviews_recipe_id_list'])

In [36]:
reviewed_recipes_list = [a for a in reviewed]

In [37]:
len(reviewed_recipes_list)

815

In [39]:
reviewed_recipes_list = [a[u'reviews_recipe_id_list'] for a in reviewed_recipes_list]

In [40]:
reviewed_recipes_list = [item for sublist in reviewed_recipes_list for item in sublist]

In [45]:
len(reviewed_recipes_list)

4162

In [47]:
len(set(reviewed_recipes_list))

3472

In [41]:
reviewed_recipes_counter = collections.Counter(reviewed_recipes_list)

In [42]:
reviewed_recipes_counter.most_common(10)

[(u'162760', 10),
 (u'56927', 8),
 (u'6820', 8),
 (u'10402', 7),
 (u'14231', 7),
 (u'9023', 7),
 (u'26317', 7),
 (u'17066', 7),
 (u'16066', 6),
 (u'8847', 6)]

In [43]:
sum(reviewed_recipes_counter.values())

4162

In [46]:
len(reviewed_recipes_counter)

3472

In [50]:
# get list of all 'made-it' recipes
madeits =  members_coll.find({"member_ID": {"$exists": True}}, ['madeits_recipe_id_list'])

In [51]:
madeits_recipes_list = [a['madeits_recipe_id_list'] for a in madeits]

In [52]:
madeits_recipes_list = [item for sublist in madeits_recipes_list for item in sublist]

In [53]:
madeits_recipes_counter = collections.Counter(madeits_recipes_list)

In [54]:
len(madeits_recipes_counter)

3399

In [55]:
sum(madeits_recipes_counter.values())

4398

In [56]:
madeits_recipes_counter.most_common(10)

[(u'10402', 10),
 (u'26317', 10),
 (u'56927', 10),
 (u'23600', 9),
 (u'162760', 8),
 (u'9023', 8),
 (u'51283', 7),
 (u'7565', 7),
 (u'14231', 7),
 (u'10813', 7)]

In [63]:
recipes_list = reviewed_recipes_counter.keys() + \
     madeits_recipes_counter.keys() + \
     favorite_recipes_counter.keys() 

In [64]:
len(recipes_list)

13976

In [68]:
recipes_orderedset = collections.OrderedDict.fromkeys(recipes_list)

In [69]:
len(recipes_orderedset)

9782

In [72]:
with open('recipes_orderedset.pkl', 'wb') as f: 
    pickle.dump(recipes_orderedset, f)